In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import time
import pyautogui

import os
import threading

In [ ]:
def tshark_thread(file_name, filt1, filt2):
    os.system(f'tshark -i 4 -w {file_name} -f "(host {filt1}) or (host {filt2})"')
    
def visit_thread(url):
    os.system('msedge --start-maximized ' + url)

In [ ]:
url_path = '/msedge/2tab-url.csv'
out_path = '/msedge/2tab-traffic/'


count_url = 100
count_repeat = 10
count_batch = 10

count = 0

df_url = pd.read_csv(url_path, header=None)

for C in range(0, int(count_url / count_batch)):

    for B in range(count_repeat):

        for A in range(C * count_batch, C * count_batch + count_batch):
        
#             if A + 1 >= C * count_batch + count_batch:
#                 break

            url1 = df_url.iloc[A, 1]
            url2 = df_url.iloc[A, 3]
           
            web1=df_url.iloc[A, 0]
            web2=int(df_url.iloc[A, 2])

            count += 1
            print('\r                                                                 ', end='')
            print('\rcapturing:', A, '-', B, end='  ')
            print(str(count) + '/' + str(count_url * count_repeat), end='  ')
            print(url1, url2,  end='  ')

            t1 = threading.Thread(target=tshark_thread, 
                                  args=(out_path + f"{web1}-{web2}-{B}.pcap", url1, url2))
            t1.start()
            
            time.sleep(4)
            
            t2 = threading.Thread(target=visit_thread, 
                                  args=(url1,))
            t2.start()

            time.sleep(8)
            loading_time = 0
            while(pyautogui.pixel(71,52) == (18,18,18)):
                time.sleep(1)
                loading_time += 1
                if loading_time >= 20:
                    break
                    
            t3 = threading.Thread(target=visit_thread, 
                                  args=(url2,))
            t3.start()
            time.sleep(8)
            loading_time = 0                                      
            while(pyautogui.pixel(71,52) == (18,18,18)):
                time.sleep(1)
                loading_time += 1
                
                if loading_time >= 20:
                    break
                    
            time.sleep(2)
            
            pyautogui.screenshot(out_path + f"{web1}-{web2}-{B}.jpg")
            time.sleep(1)
            os.system('taskkill /F /IM tshark.exe')
            os.system('taskkill /F /IM msedge.exe')

            time.sleep(1)
    
print('\ndone!')


In [ ]:
import os
os.system('taskkill /F /IM msedge.exe')
os.system('taskkill /F /IM tshark.exe')